In [23]:
import requests
import time
import csv
import random

In [24]:
categories = {"categories":[{"cat_key":"OVERALL","name":"Overall"},{"cat_key":"APPLICATION","name":"All apps"},{"cat_key":"GAME","name":"All games"},{"cat_key":"ART_AND_DESIGN","name":"Art & Design"},{"cat_key":"AUTO_AND_VEHICLES","name":"Auto & Vehicles"},{"cat_key":"BEAUTY","name":"Beauty"},{"cat_key":"BOOKS_AND_REFERENCE","name":"Books & Reference"},{"cat_key":"BUSINESS","name":"Business"},{"cat_key":"COMICS","name":"Comics"},{"cat_key":"COMMUNICATION","name":"Communication"},{"cat_key":"DATING","name":"Dating"},{"cat_key":"EDUCATION","name":"Education"},{"cat_key":"ENTERTAINMENT","name":"Entertainment"},{"cat_key":"EVENTS","name":"Events"},{"cat_key":"FINANCE","name":"Finance"},{"cat_key":"FOOD_AND_DRINK","name":"Food & Drink"},{"cat_key":"HEALTH_AND_FITNESS","name":"Health & Fitness"},{"cat_key":"HOUSE_AND_HOME","name":"House & Home"},{"cat_key":"LIFESTYLE","name":"Lifestyle"},{"cat_key":"MAPS_AND_NAVIGATION","name":"Maps & Navigation"},{"cat_key":"MEDICAL","name":"Medical"},{"cat_key":"MUSIC_AND_AUDIO","name":"Music & Audio"},{"cat_key":"NEWS_AND_MAGAZINES","name":"News & Magazines"},{"cat_key":"PARENTING","name":"Parenting"},{"cat_key":"PERSONALIZATION","name":"Personalization"},{"cat_key":"PHOTOGRAPHY","name":"Photography"},{"cat_key":"PRODUCTIVITY","name":"Productivity"},{"cat_key":"SHOPPING","name":"Shopping"},{"cat_key":"SOCIAL","name":"Social"},{"cat_key":"SPORTS","name":"Sports"},{"cat_key":"TOOLS","name":"Tools"},{"cat_key":"TRAVEL_AND_LOCAL","name":"Travel & Local"},{"cat_key":"VIDEO_PLAYERS","name":"Video Players & Editors"},{"cat_key":"WEATHER","name":"Weather"},{"cat_key":"LIBRARIES_AND_DEMO","name":"Libraries & Demo"},{"cat_key":"GAME_ARCADE","name":"Arcade"},{"cat_key":"GAME_PUZZLE","name":"Puzzle"},{"cat_key":"GAME_CARD","name":"Cards"},{"cat_key":"GAME_CASUAL","name":"Casual"},{"cat_key":"GAME_RACING","name":"Racing"},{"cat_key":"GAME_SPORTS","name":"Sport Games"},{"cat_key":"GAME_ACTION","name":"Action"},{"cat_key":"GAME_ADVENTURE","name":"Adventure"},{"cat_key":"GAME_BOARD","name":"Board"},{"cat_key":"GAME_CASINO","name":"Casino"},{"cat_key":"GAME_EDUCATIONAL","name":"Educational"},{"cat_key":"GAME_MUSIC","name":"Music Games"},{"cat_key":"GAME_ROLE_PLAYING","name":"Role Playing"},{"cat_key":"GAME_SIMULATION","name":"Simulation"},{"cat_key":"GAME_STRATEGY","name":"Strategy"},{"cat_key":"GAME_TRIVIA","name":"Trivia"},{"cat_key":"GAME_WORD","name":"Word Games"},{"cat_key":"ANDROID_WEAR","name":"Android Wear"},{"cat_key":"FAMILY","name":"Family All Ages"},{"cat_key":"FAMILY_UNDER_5","name":"Family Ages 5 & Under"},{"cat_key":"FAMILY_6_TO_8","name":"Family Ages 6-8"},{"cat_key":"FAMILY_9_AND_UP","name":"Family Ages 9 & Up"},{"cat_key":"FAMILY_ACTION","name":"Family Action"},{"cat_key":"FAMILY_BRAINGAMES","name":"Family Brain Games"},{"cat_key":"FAMILY_CREATE","name":"Family Create"},{"cat_key":"FAMILY_EDUCATION","name":"Family Education"},{"cat_key":"FAMILY_MUSICVIDEO","name":"Family Music & Video"},{"cat_key":"FAMILY_PRETEND","name":"Family Pretend Play"}]}

In [25]:
category_keys = [cat["cat_key"] for cat in categories["categories"]]

In [26]:
def get_top_free_applications(categories):
    def per_category_top_free(category):
        waiting_urls = set()
        url = 'http://localhost:3000/api/apps/?collection=topselling_free&category={}&lang=en'.format(category)
        #top free applications
        response = requests.get(url)
        if response.status_code == 200:
            for app in response.json()["results"]:
                waiting_urls.add(app["url"])
        else:
            print("Request Error")
        return waiting_urls
    all_top_free_apps = set()
    for category in categories:
        all_top_free_apps.update(per_category_top_free(category))
    return all_top_free_apps

In [27]:
def get_app_ids(top_free_apps):
    waiting_urls = list(top_free_apps)
    random.shuffle(waiting_urls)
    waiting_urls_unique_check = set([url for url in waiting_urls])
    app_ids = set()

    with open("app_ids_record_audio.txt", "w", buffering=1) as target:
        counter = 0
        start_time = time.time()
        while (len(app_ids) + len(waiting_urls)) < 100000:
            if counter % 10 == 0:
                elapsed_time = time.time() - start_time
                print("Total number of apps", len(app_ids) + len(waiting_urls))
                print("Collected app ids", len(app_ids))
                print("Elapsed time up to now is {}".format(elapsed_time))
            try:
                url = waiting_urls.pop()
                waiting_urls_unique_check.remove(url)
            except KeyError:
                print("All linked applications are traversed")
                break

            #add apk id if it is free, popular, and has longer description than 100 characters
            response = requests.get(url)
            if response.status_code == 200:
                json = response.json()
                if json["minInstalls"] > 10000 and json["priceText"] == "Free" and len(json["description"]) > 500:
                    if url.split('/')[-1] not in app_ids:
                        response = requests.get(url+"/permissions")
                        if response.status_code == 200:
                            json = response.json()
                            exist = False
                            for line in json["results"]:
                                if "record audio" in line["permission"]:
                                    app_ids.add(url.split('/')[-1] )
                                    target.write(url.split('/')[-1]+"\n")

            #add similar app urls
            response = requests.get(url + '/similar')
            if response.status_code == 200:
                for app in response.json()["results"]:
                    if app["appId"] not in app_ids:
                        if app["url"] not in waiting_urls_unique_check:
                            waiting_urls_unique_check.add(app["url"])
                            waiting_urls.append(app["url"])
            random.shuffle(waiting_urls)
            counter += 1

        for url in waiting_urls:
            response = requests.get(url)
            if response.status_code == 200:
                json = response.json()
                if json["minInstalls"] > 10000 and json["priceText"] == "Free" and len(json["description"]) > 500:
                    if url.split('/')[-1] not in app_ids:
                        app_ids.add(url.split('/')[-1] )
                        target.write(url.split('/')[-1]+"\n")

In [28]:
top_free_apps = get_top_free_applications(category_keys)

Request Error
Request Error
Request Error
Request Error


In [29]:
from threading import Thread, Condition
import time
import random

waiting_urls = list(top_free_apps)
random.shuffle(waiting_urls)
waiting_urls_unique_check = set([url for url in waiting_urls])
app_ids = set()
start_time = time.time()
similarity_candidates = []
condition = Condition()

class SimilarityThread(Thread):
    def run(self):
        nums = range(5)
        global waiting_urls, waiting_urls_unique_check, similarity_candidates
        while True:
            condition.acquire()
            random.shuffle(similarity_candidates)
            for i in range(3):
                url = similarity_candidates.pop()
                response = requests.get(url + '/similar')
                if response.status_code == 200:
                    for app in response.json()["results"]:
                        if app["appId"] not in app_ids:
                            if app["url"] not in waiting_urls_unique_check:
                                waiting_urls_unique_check.add(app["url"])
                                waiting_urls.append(app["url"])
                else:
                    print("SimilarityThread ", url)
            condition.notify()
            condition.release()
            time.sleep(random.random())


class AppIdThread(Thread):
    def run(self):
        global waiting_urls, similarity_candidates, app_ids, start_time
        while True:
            condition.acquire()
            if not waiting_urls:
                print ("Nothing in queue, consumer is waiting")
                condition.wait()
                print ("Producer added something to queue and notified the consumer")
            
            elapsed_time = time.time() - start_time
            print("Total number of apps", len(app_ids) + len(waiting_urls))
            print("Collected app ids", len(app_ids))
            print("Elapsed time up to now is {}".format(elapsed_time))
            
            with open("app_ids_record_audio.txt", "a", buffering=1) as target:
                while waiting_urls:
                    url = waiting_urls.pop()
                    waiting_urls_unique_check.remove(url)
                    #add apk id if it is free, popular, and has longer description than 100 characters
                    response = requests.get(url)
                    if response.status_code == 200:
                        json = response.json()
                        if json["minInstalls"] > 10000 and json["priceText"] == "Free" and len(json["description"]) > 500:
                            if url.split('/')[-1] not in app_ids:
                                response = requests.get(url+"/permissions")
                                if response.status_code == 200:
                                    json = response.json()
                                    exist = False
                                    for line in json["results"]:
                                        if "record audio" in line["permission"]:
                                            app_ids.add(url.split('/')[-1] ) 
                                            similarity_candidates.append(url)
                                            target.write(url.split('/')[-1]+"\n")
                    else:
                        print("AppIdThread ", url)
                                    
            condition.release()
            time.sleep(random.random())
    
    
AppIdThread().start()
SimilarityThread().start()

Total number of apps 3173
Collected app ids 0
Elapsed time up to now is 0.0011150836944580078
Total number of apps 615
Collected app ids 480
Elapsed time up to now is 1602.534376859665
Total number of apps 654
Collected app ids 535
Elapsed time up to now is 1665.1653456687927
Total number of apps 683
Collected app ids 565
Elapsed time up to now is 1723.2306110858917
Total number of apps 696
Collected app ids 585
Elapsed time up to now is 1783.0470950603485
Total number of apps 719
Collected app ids 607
Elapsed time up to now is 1838.641252040863
Total number of apps 739
Collected app ids 629
Elapsed time up to now is 1894.8913488388062
Total number of apps 747
Collected app ids 652
Elapsed time up to now is 1947.387506723404
Total number of apps 716
Collected app ids 662
Elapsed time up to now is 1995.7816710472107
Total number of apps 748
Collected app ids 671
Elapsed time up to now is 2022.2583050727844
Total number of apps 784
Collected app ids 680
Elapsed time up to now is 2062.050